# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np

import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer

from sqlalchemy import create_engine
import re

from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support, make_scorer, accuracy_score, f1_score, fbeta_score, classification_report

import pickle

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\saiya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saiya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponsetbl', con=engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # Remove punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    
    # lemmatization
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for colnm in y_test.columns:
    print('FEATURE: {}\n'.format(colnm))
    print(classification_report(y_test[colnm],y_pred_pd[colnm]))

FEATURE: related

              precision    recall  f1-score   support

           0       0.60      0.35      0.45      1470
           1       0.83      0.93      0.88      5040
           2       0.58      0.32      0.41        44

    accuracy                           0.80      6554
   macro avg       0.67      0.53      0.58      6554
weighted avg       0.78      0.80      0.78      6554

FEATURE: request

              precision    recall  f1-score   support

           0       0.88      0.98      0.93      5386
           1       0.84      0.36      0.51      1168

    accuracy                           0.87      6554
   macro avg       0.86      0.67      0.72      6554
weighted avg       0.87      0.87      0.85      6554

FEATURE: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accuracy                           1.00      6554
   macro avg       0.5

In [14]:
# Model results in dataframe format
def get_results(y_test, y_pred):
    report = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])
    num = 0
    for colnm in y_test.columns:
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[colnm], y_pred[:,num], average='weighted')
        report.at[num+1, 'Category'] = colnm
        report.at[num+1, 'f_score'] = f_score
        report.at[num+1, 'precision'] = precision
        report.at[num+1, 'recall'] = recall
        num += 1
    print('Aggregated f_score:', report['f_score'].mean())
    print('Aggregated precision:', report['precision'].mean())
    print('Aggregated recall:', report['recall'].mean())
    print('Accuracy:', np.mean(y_test.values == y_pred))
    return report

In [15]:
report = get_results(y_test, y_pred)
report

Aggregated f_score: 0.927360826503317
Aggregated precision: 0.9302653401590574
Aggregated recall: 0.9416090258705455
Accuracy: 0.9416090258705455


,Category,f_score,precision,recall
1,related,0.776649,0.77591,0.797528
2,request,0.852473,0.869851,0.873818
3,offer,0.993596,0.991474,0.995728
4,aid_related,0.718044,0.734099,0.728868
5,medical_help,0.888799,0.894929,0.918523
6,medical_products,0.934813,0.945466,0.953464
7,search_and_rescue,0.958381,0.960897,0.971163
8,security,0.971937,0.963109,0.980928
9,military,0.954402,0.961448,0.967501
10,child_alone,1,1,1


### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
parameters = {'vect__ngram_range': ((1, 1), (1, 2))
              ,'vect__max_df': (1.0,2.0)
              #,'clf__estimator__max_depth': [10,20,None],
              #,'clf__estimator__min_samples_leaf':[2, 5]
             }

cv = GridSearchCV(estimator=pipeline,
            param_grid=parameters,
            verbose=3,
            cv=3
            ,scoring='f1_weighted'
            ,n_jobs = -1
                 )

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
report2 = get_results(y_test, y_pred)
report2

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed:   59.9s remaining:  5.0min
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:  2.5min remaining:  1.8min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  3.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  3.2min finished


Aggregated f_score: 0.927721728091413
Aggregated precision: 0.9315116179973225
Aggregated recall: 0.9415666429322214
Accuracy: 0.9415666429322213


,Category,f_score,precision,recall
1,related,0.762026,0.758984,0.785017
2,request,0.859846,0.872349,0.877785
3,offer,0.993596,0.991474,0.995728
4,aid_related,0.710537,0.734563,0.724748
5,medical_help,0.887837,0.897374,0.918828
6,medical_products,0.935706,0.945741,0.953769
7,search_and_rescue,0.959262,0.967022,0.971773
8,security,0.972089,0.963115,0.981233
9,military,0.952278,0.956271,0.966433
10,child_alone,1,1,1


In [20]:
cv.best_estimator_

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [25]:
# AdaBoost classifier

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

pipeline.fit(X_train.as_matrix(), y_train.as_matrix())
y_pred = pipeline.predict(X_test)
report3 = get_results(y_test, y_pred)
report3

Aggregated f_score: 0.9400721387214062
Aggregated precision: 0.9393948623401845
Aggregated recall: 0.9474621096531382
Accuracy: 0.947462109653138


,Category,f_score,precision,recall
1,related,0.745178,0.746732,0.780287
2,request,0.881518,0.881797,0.889075
3,offer,0.993138,0.99147,0.994812
4,aid_related,0.748685,0.756478,0.754654
5,medical_help,0.911025,0.910643,0.925542
6,medical_products,0.954041,0.953795,0.960482
7,search_and_rescue,0.966197,0.967179,0.973604
8,security,0.972208,0.967205,0.979402
9,military,0.96681,0.966173,0.971468
10,child_alone,1,1,1


### 9. Export your model as a pickle file

In [27]:
pickle.dump(cv, open('model.pkl', 'wb'))
pickle.dump(pipeline, open('classifier.pkl', 'wb'))
# load the model from disk
# loaded_model = pickle.load(open('model.pkl', 'rb'))
# result = loaded_model.score(X_test, Y_test)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.